# 1.Import and describe dataset

In [1]:
import os
import pandas as pd

citypath= os.path.expanduser('/data/tianchi/data/CityData.csv')
Insitupath=os.path.expanduser("/data/tianchi/data/In_situMeasurementforTraining_201712.csv")
Trainpath=os.path.expanduser("/data/tianchi/data/ForecastDataforTraining_201712.csv")

In [2]:
city=pd.read_csv(citypath)
# city location 0 represent london is the starting point for all balloons
city

,cid,xid,yid
0,0,142,328
1,1,84,203
2,2,199,371
3,3,140,234
4,4,236,241
5,5,315,281
6,6,358,207
7,7,363,237
8,8,423,266
9,9,125,375


In [3]:
Insitu=pd.read_csv(Insitupath)
Insitu.head() # Real-time wind value 

,xid,yid,date_id,hour,wind
0,1,1,1,3,12.8
1,1,2,1,3,12.8
2,1,3,1,3,12.6
3,1,4,1,3,12.1
4,1,5,1,3,11.6


Insitu data represents the data generated and collected from the real world. 
Xid and Yid forms a 548 * 421 cell blocks. 
Hour range from 3 to 20 , which means each day the balloon has 18 hours to fly across the map
Day_id shows that we have totally 5 days, each day has one mission for navigating 10 balloons from city id 0 to the other 10 cities.


In [4]:
Insitu.describe()

,xid,yid,date_id,hour,wind
count,2.076372e+07,2.076372e+07,2.076372e+07,2.076372e+07,2.076372e+07
mean,2.745000e+02,2.110000e+02,3.000000e+00,1.150000e+01,1.129610e+01
std,1.581937e+02,1.215319e+02,1.414214e+00,5.188128e+00,5.491615e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00
25%,1.377500e+02,1.060000e+02,2.000000e+00,7.000000e+00,7.360000e+00
50%,2.745000e+02,2.110000e+02,3.000000e+00,1.150000e+01,1.060000e+01
75%,4.112500e+02,3.160000e+02,4.000000e+00,1.600000e+01,1.490000e+01
max,5.480000e+02,4.210000e+02,5.000000e+00,2.000000e+01,4.830000e+01


In [5]:
Train=pd.read_csv(Trainpath)
Train.head()

,xid,yid,date_id,hour,model,wind
0,1,1,1,3,1,13.8
1,1,1,1,3,2,12.8
2,1,1,1,3,3,10.4
3,1,1,1,3,4,11.1
4,1,1,1,3,5,13.2


Every day we have 10 models to forcast the weather and most of the models are 96-97 percent correct. We would like to predict the weather using the 10 models.

In [6]:
Train.describe()

,xid,yid,date_id,hour,model,wind
count,1.869645e+08,1.869645e+08,1.869645e+08,1.869645e+08,1.869645e+08,1.869645e+08
mean,2.743718e+02,2.109989e+02,2.778860e+00,1.100049e+01,5.500000e+00,1.185026e+01
std,1.582618e+02,1.215317e+02,1.315278e+00,5.162767e+00,2.872281e+00,5.516854e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00
25%,1.370000e+02,1.060000e+02,2.000000e+00,7.000000e+00,3.000000e+00,7.970000e+00
50%,2.740000e+02,2.110000e+02,3.000000e+00,1.100000e+01,5.000000e+00,1.120000e+01
75%,4.110000e+02,3.160000e+02,4.000000e+00,1.500000e+01,8.000000e+00,1.540000e+01
max,5.480000e+02,4.210000e+02,5.000000e+00,2.000000e+01,1.000000e+01,4.930000e+01


# 2.Model Selection
We would like to use logistic regression model for the training data. As we are only interested in whether the wind will be over 15, binary classification will be the toolkits.
However Logistic models requires the explanatory variable X to be uncorrelately. The ten models output are highly correlated thus orthogonal decomposition is needed for data shrinkage. In this project, we would like to use Principal Component Analysis. 

## 2.1 reshape the data to fit the model

In [13]:
sampletrain = Train[Train.date_id==1]# subset only one day one hour

In [49]:
sampletable=sampletrain.pivot_table(values="wind",index=["xid","yid","date_id","hour"],columns="model")
sampletable.head(10)

model                    1      2      3      4     5     6      7     8   \
xid yid date_id hour                                                        
1   1   1       3     13.80  12.80  10.40  11.10  13.2  13.1  12.60  11.2   
                4     12.20  12.00  11.50  11.60  11.8  12.1  11.60  11.0   
                5     13.50  13.40  12.20  11.80  12.2  13.5  12.20  11.1   
                6     12.70  15.30  15.70  15.00  15.7  16.6  16.40  14.5   
                7     11.00  11.10  11.40  11.30  11.5  11.0  10.20  10.7   
                8     12.50  12.30  12.20  12.80  13.3  12.8  12.80  11.2   
                9     11.00  12.30  11.30  12.40  12.5  12.3  11.90  12.1   
                10     9.89   9.64   9.78   8.94  11.1  10.3   9.29  11.5   
                11    10.80  11.20  10.50  10.90  11.0  12.5   9.67  11.4   
                12    10.00  10.30  10.40   9.89  10.3  11.1  10.20   9.7   

model                    9      10  
xid yid date_id hour                
1   1   1       3     14.10  12.80  
                4     10.90  12.00  
                5     11.30  13.50  
                6     13.90  14.60  
                7      9.75  12.00  
                8     11.20  12.20  
                9     11.10  12.10  
                10     8.63   9.43  
                11    10.90  11.70  
                12    10.00  10.00

In [50]:
sampleinsitu=Insitu[Insitu.date_id==1]

In [51]:
sampleinsitu.head()

,xid,yid,date_id,hour,wind
0,1,1,1,3,12.8
1,1,2,1,3,12.8
2,1,3,1,3,12.6
3,1,4,1,3,12.1
4,1,5,1,3,11.6


In [52]:
sampletrue=sampleinsitu.set_index(['xid','yid','date_id','hour'])

In [53]:
samplemodel=sample.set_index(["xid","yid","date_id","hour"])

In [54]:
sampletrue.shape

(4152744, 1)

In [56]:
samplemodel.shape

(4152744, 10)

In [57]:
samplemodel["ture"]=sampletrue

In [58]:
samplemodel.head()

model                    1     2     3     4     5     6     7     8      9  \
xid yid date_id hour                                                          
1   1   1       3     13.8  12.8  10.4  11.1  13.2  13.1  12.6  11.2  14.10   
                4     12.2  12.0  11.5  11.6  11.8  12.1  11.6  11.0  10.90   
                5     13.5  13.4  12.2  11.8  12.2  13.5  12.2  11.1  11.30   
                6     12.7  15.3  15.7  15.0  15.7  16.6  16.4  14.5  13.90   
                7     11.0  11.1  11.4  11.3  11.5  11.0  10.2  10.7   9.75   

model                   10  ture  
xid yid date_id hour              
1   1   1       3     12.8  12.8  
                4     12.0  11.7  
                5     13.5  12.4  
                6     14.6  15.3  
                7     12.0  11.3

In [63]:
import pickle
import tempfile

# does not create the file, just makes a pathname
path = os.path.expanduser('/data/tianchi/data/day1')
path

'/data/tianchi/data/day1'

In [64]:
# 'bw' - 'binary write'
with open(path, 'bw') as f:
    pickle.dump(samplemodel, f)

In [61]:
import pickle
with open(path, 'br') as f:
    samplemodel2=pickle.load(f)
samplemodel2.head()

In [62]:
samplemodel2.head()

model                    1     2     3     4     5     6     7     8      9  \
xid yid date_id hour                                                          
1   1   1       3     13.8  12.8  10.4  11.1  13.2  13.1  12.6  11.2  14.10   
                4     12.2  12.0  11.5  11.6  11.8  12.1  11.6  11.0  10.90   
                5     13.5  13.4  12.2  11.8  12.2  13.5  12.2  11.1  11.30   
                6     12.7  15.3  15.7  15.0  15.7  16.6  16.4  14.5  13.90   
                7     11.0  11.1  11.4  11.3  11.5  11.0  10.2  10.7   9.75   

model                   10  ture  
xid yid date_id hour              
1   1   1       3     12.8  12.8  
                4     12.0  11.7  
                5     13.5  12.4  
                6     14.6  15.3  
                7     12.0  11.3

In [7]:
Train=Train.pivot_table(values="wind",index=["xid","yid","date_id","hour"],columns="model")

In [8]:
true=Insitu.set_index(['xid','yid','date_id','hour'])


In [11]:
true.head()

wind
xid yid date_id hour      
1   1   1       3     12.8
    2   1       3     12.8
    3   1       3     12.6
    4   1       3     12.1
    5   1       3     11.6

In [10]:
Train.head()

model                   1     2     3     4     5     6     7     8      9   \
xid yid date_id hour                                                          
1   1   1       3     13.8  12.8  10.4  11.1  13.2  13.1  12.6  11.2  14.10   
                4     12.2  12.0  11.5  11.6  11.8  12.1  11.6  11.0  10.90   
                5     13.5  13.4  12.2  11.8  12.2  13.5  12.2  11.1  11.30   
                6     12.7  15.3  15.7  15.0  15.7  16.6  16.4  14.5  13.90   
                7     11.0  11.1  11.4  11.3  11.5  11.0  10.2  10.7   9.75   

model                   10  
xid yid date_id hour        
1   1   1       3     12.8  
                4     12.0  
                5     13.5  
                6     14.6  
                7     12.0

In [13]:
Train.shape

(18696454, 10)

In [14]:
true.shape 

(20763720, 1)

In [15]:
Train["true"]=true


In [16]:
Train.head()

model                    1     2     3     4     5     6     7     8      9  \
xid yid date_id hour                                                          
1   1   1       3     13.8  12.8  10.4  11.1  13.2  13.1  12.6  11.2  14.10   
                4     12.2  12.0  11.5  11.6  11.8  12.1  11.6  11.0  10.90   
                5     13.5  13.4  12.2  11.8  12.2  13.5  12.2  11.1  11.30   
                6     12.7  15.3  15.7  15.0  15.7  16.6  16.4  14.5  13.90   
                7     11.0  11.1  11.4  11.3  11.5  11.0  10.2  10.7   9.75   

model                   10  true  
xid yid date_id hour              
1   1   1       3     12.8  12.8  
                4     12.0  11.7  
                5     13.5  12.4  
                6     14.6  15.3  
                7     12.0  11.3

In [17]:
Train.shape # it seems we neglect some data from insitu dataset

(18696454, 11)

In [19]:
import pickle
shapepath="/data/tianchi/data/Train"
with open(shapepath, 'bw') as f:
    pickle.dump(Train,f)
